In [2]:
# 서울시 홈페이지 접속
# 서울소식 버튼 클릭
# 게시글 크롤링
# 1페이지~5페이지
# 각각의 페이지 내 게시글 제목 & 게시글 클릭 이동 페이지 URL
# URL 이동 후 게시글을 업로드한 부서 & 게시 날짜 찾아오기
# 어느 부서가 가장 많은 게시물을 업로드했는지 시각화
# 어느 날짜에 게시물들이 많이 업로드됐는지

In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

import time
import pandas as pd
from urllib.parse import urljoin

service = Service(ChromeDriverManager().install())
options = Options()
# options.add_argument("--headless")
# options.add_argument("--disable-gpu")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--window-size=1920x1080")
options.add_argument("--start-maximized")
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36")
options.add_argument("--lang=ko_KR")

driver = webdriver.Chrome(service=service, options=options)
wait = WebDriverWait(driver, 10)

# 목록 제목 &링크값을 수집하는 함수를 선언한다
def collect_list_links_on_current_page() :
    base = driver.current_url # current_url 은 https://www.seoul.go.kr/realmnews/in/list.do
    items = driver.find_elements(By.CSS_SELECTOR, "div.news-lst div.item > a")

    links = []
    for item in items :
        title = item.find_element(By.CSS_SELECTOR, "em.subject").text.strip()
        href = item.get_attribute("href")
        url = urljoin(base, href)
        links.append({"list_title": title, "url": url})
    return links    

# 각 페이지당 부서명, 작성 날짜를 찾아오는 함수를 선언한다
def scape_detail_fields(article_url) :
    driver.get(article_url)

    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#view_top")))

    title = driver.find_element(By.CSS_SELECTOR, "#View_top h3").text.strip()
    
    dept_spans = driver.find_elements(By.CSS_SELECTOR, "#view_top dd.dept span")
    dept = " - ".join([s.text.strip() for s in dept_spans if s.text.strip()]) 
    modified_date = driver.find_element(By.CSS_SELECTOR, "#view_top dd.date").text.strip()
    return title, dept, modified_date

try :
    driver.get("https://www.seoul.go.kr/main/index.jsp")
    time.sleep(2)
    
    wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a[href='/realmnews/in/list.do']"))).click()
    
    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.news-lst div.item > a")))

    max_page = 5
    collected_links = []

    for page in range(1, max_page + 1) :
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(0.5)

        # 선언한 함수를 호출하는 지점
        collected_links.extend(collect_list_links_on_current_page())

        print(f"[목록수집] {page}페이지 누적 링크 {len(collected_links)}개")

        if page == max_page :
            break

        next_btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, f"#paging_main ul.pagination a[data-page='{page+1}']")))
        next_btn.click()

        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(0.3)
        
    tmp_df = pd.DataFrame(collected_links).drop_duplicates(subset=["url"]).reset_index(drop=True)
    link_list = tmp_df["url"].to_list()

    rows = []
    for i, link in enumerate(link_list, start=1) :
        try :
            title, dept, modified_date = scape_detail_fields(link)
            rows.append({
                "title": title, 
                "url" : link,
                "dept" : dept,
                "modified_date": modified_date
            })
            print(f"[상세수집] {i}/{len(link_list)} 완료")
        except Exception as e :
            rows.append({
                "title": "", 
                "url": link,
                "dept": "",
                "modified_date":"",
                "error": str(e)
            })
            print(f"[상세실패] {i}/{len(link_list)} {link} -> {e}")

        time.sleep(0.3)
    df = pd.DataFrame(rows)
    filename = "seoul_news_p1_to_p5_with_dept_dat.csv"
    df.to_csv(filename, index=False, encoding="utf-8-sig")

    print(f"CSV파일 저장 완료 : {filename}")
        
finally : 
    driver.quit()

[목록수집] 1페이지 누적 링크 10개
[목록수집] 2페이지 누적 링크 20개
[목록수집] 3페이지 누적 링크 30개
[목록수집] 4페이지 누적 링크 40개
[목록수집] 5페이지 누적 링크 50개
[상세실패] 1/50 https://news.seoul.go.kr/welfare/archives/576396 -> Message: no such element: Unable to locate element: {"method":"css selector","selector":"#View_top h3"}
  (Session info: chrome=144.0.7559.60); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#nosuchelementexception
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x92ccf3
	0x92cd34
	0x74b1e0
	0x79561a
	0x7958bb
	0x7d78e2
	0x7b7f54
	0x7d51f3
	0x7b7ca6
	0x789409
	0x78a1c4
	0xb9a7b4
	0xb95db7
	0xbb379d
	0x946ca8
	0x94e87d
	0x935708
	0x9358d2
	0x91f03a
	0x76c0fcc9
	0x77e982ae
	0x77e9827e

[상세실패] 2/50 https://news.seoul.go.kr/citybuild/archives/530758 -> Message: no such element: Unable to locate element: {"method":"css selector","selector":"#View_top h3"}
  (Session info: chrome=144.0.7559.60); For documentation on this error, pleas

# 작성 순서 버전

from webdriver_manager.chrome import ChromeDriverManager

ChromeDriverManager().install()
from selenium.webdriver.chrome.service import Service
>> service = Service(ChromeDriverManager().install())

from selenium import webdriver -> 크롤러 찾아와라, webdriver 크롤러 역할

webdriver.Chrome(service=service) -> 앞의 service 는 param, 뒤의 service는 위에 설정한 변수

from selenium.webdriver.chrome.options import Options

options = Options()

webdriver.Chrome(service=service, options=options) -> 앞의 options 는 param, 뒤의 options는 위에 설정한 변수

options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--window-size=1920x1080")
options.add_argument("--start-maximized")
options.add_argument("--user-agent=")
options.add_argument("--lang=ko_KR")

변수 설정(크롤러) driver = webdriver.Chrome(service=service, options=options)

크롤러야, "url" 갔다와 driver.get("https://www.seoul.go.kr/main/index.jsp")

import time
time.sleep(2)

a href="/realmnews/in/list.do" --> 서울소식 '검사'에서 가져온 값
from selenium.webdriver.common.by import By
먼저 By.CSS_SELECTOR, "a[href='/realmnews/in/list.do']" 작성

from selenium.webdriver.support.ui import WebDriverWait 
wait = WebDriverWait(driver, 10) driver라 10초 기다려, 근데 그 전에 실행할 수 있으면 해 

wait.until() 추가해서
(By.CSS_SELECTOR, "a[href='/realmnews/in/list.do']") 업데이트

from selenium.webdriver.support import expected_conditions as EC --> 이름 기니까 EC로 지정
wait.until(EC.element_to_be_clickable())
(By.CSS_SELECTOR, "a[href='/realmnews/in/list.do']") 업데이트
>> wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a[href='/realmnews/in/list.do']"))).click() 

게시물 제목 검사
div.news-lst div.item > a

wait.until(EC.presence_of_all_elements_located)
(By.CSS_SELECTOR, "div.news-lst div.item > a")
>> wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.news-lst div.item > a")))


try :
    driver.get("https://www.seoul.go.kr/main/index.jsp")
    time.sleep(2)
    
    wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a[href='/realmnews/in/list.do']"))).click()
    
    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.news-lst div.item > a")))


finally : 
    driver.quit()


max_page = 5 --> 5페이지까지 반복해서
collected_links = [] --> 리스트 만들어라

for page in range(1, max_page + 1) :
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") --> 다음 페이지 넘어가도록 스크롤해라, (자바)스크립트는 ; 로 마무리

def collect_list_links_on_current_page() --> 해당 목록당 링크를 현재 페이지에 수집하기 위한 함수(def). 위험 분산

collected_links.extend(collect_list_links_on_current_page())

def collect_list_links_on_current_page() :
    base = driver.current_url # current_url 은 https://www.seoul.go.kr/realmnews/in/list.do
    items = driver.find_elements(By.CSS_SELECTOR, "div.news-lst div.item > a")

    for item in items :
        title = item.find_element(By.CSS_SELECTOR, "em.subject").text.strip()
        href = item.get_attribute("href")

from urllib.parse import urljoin

    links = []
    for item in items :
        title = item.find_element(By.CSS_SELECTOR, "em.subject").text.strip()
        href = item.get_attribute("href")
        url = urljoin(base, href)
        links.append({"list_title": title, "url": url})
    return links  

print(f"[목록수집] {page}페이지 누적 링크 {len(collected_links)}개")

        if page == max_page :
            break

        next_btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, f"#paging_main ul.pagination a[data-page='{page+1}']")))
        next_btn.click()

                driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(0.3)
print(collected_links)

import pandas as pd

    tmp_df = pd.DataFrame(collected_links).drop_duplicates(subset=["url"]).reset_index(drop=True)
    link_list = tmp_df["url"].to_list()

    print(tmp_df)
    print(link_list)

    for i, link in enumerate(link_list, start=1) :
        try :


# 각 페이지당 부서명, 작성 날짜를 찾아오는 함수를 선언한다
def scape_detail_fields(article_url) :
    driver.get(article_url)

    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#view_top")))

    title = driver.find_element(By.CSS_SELECTOR, "#View_top h3").text.strip()
    
    dept_spans = driver.find_elements(By.CSS_SELECTOR, "#view_top dd.dept span")
    dept = " - ".join([s.text.strip() for s in dept_spans if s.text.strip()]) 
    modified_date = driver.find_element(By.CSS_SELECTOR, "#view_top dd.date").text.strip()
    return title, dept, modified_date


    rows = []
    for i, link in enumerate(link_list, start=1) :
        try :
            title, dept, modified_date = scape_detail_fields(link)
            rows.append({
                "title": title, 
                "url" : link,
                "dept" : dept,
                "modified_date": modified_date
            })
            print(f"[상세수집] {i}/{len(link_list)} 완료")
        except Exception as e :
            rows.append({
                "title": "", 
                "url" : link,
                "dept" : "",
                "modified_date":""
                "error": str(e)
            })